In [ ]:
# =============================================
# 1. CONFIGURACIÓN INICIAL Y MANEJO DE REINICIOS
# =============================================

import os
import sys
from pathlib import Path
import subprocess
import warnings
warnings.filterwarnings('ignore')

# Función para verificar si estamos en un entorno recién reiniciado
def check_colab_restart():
    """Verifica si necesitamos reconfigurar rutas después de un reinicio"""
    try:
        # Intentar importar paquetes que instalamos
        import torch
        import librosa
        return False  # Todo está bien
    except ImportError:
        return True  # Necesitamos reconfigurar

# Directorios base (siempre se definen)
BASE_DIR = Path('/content')
OPENVOICE_DIR = BASE_DIR / 'OpenVoice'
MELOTTS_DIR = BASE_DIR / 'MeloTTS'

# Crear directorios si no existen
OPENVOICE_DIR.mkdir(exist_ok=True)
MELOTTS_DIR.mkdir(exist_ok=True)

print("="*60)
print("1. CONFIGURACIÓN DE OPENVOICE Y MELOTTS")
print("="*60)
print(f"📁 Directorio base: {BASE_DIR}")
print(f"📁 OpenVoice: {OPENVOICE_DIR}")
print(f"📁 MeloTTS: {MELOTTS_DIR}")

# Verificar si necesitamos reinstalar
if check_colab_restart():
    print("\n⚠️  Detectado posible reinicio de Colab")
    print("   Se reinstalarán las dependencias necesarias...")

1. CONFIGURACIÓN DE OPENVOICE Y MELOTTS
📁 Directorio base: /content
📁 OpenVoice: /content/OpenVoice
📁 MeloTTS: /content/MeloTTS


In [ ]:
# =============================================
# 2. INSTALACIÓN DE OPENVOICE
# =============================================

print("\n" + "="*60)
print("2. INSTALANDO OPENVOICE")
print("="*60)

# Clonar repositorio OpenVoice si no existe
if not (OPENVOICE_DIR / '.git').exists():
    print("📥 Clonando OpenVoice...")
    !git clone https://github.com/PhonelixR/OpenVoice.git "{OPENVOICE_DIR}" --quiet
else:
    print("✅ OpenVoice ya está clonado")

# Cambiar al directorio de OpenVoice
os.chdir(OPENVOICE_DIR)
print(f"📂 Directorio actual: {os.getcwd()}")

# Instalar dependencias del sistema
print("\n📦 Instalando dependencias del sistema...")
!apt-get update -qq
!apt-get install -y -qq build-essential cmake libpython3-dev ffmpeg > /dev/null 2>&1

# Instalar dependencias de Python
print("📦 Instalando dependencias de Python...")
!pip install torch torchaudio --index-url https://download.pytorch.org/whl/cpu -q
!pip install librosa soundfile transformers gradio pyworld -q
!pip install -r requirements.txt -q


2. INSTALANDO OPENVOICE
📥 Clonando OpenVoice...
📂 Directorio actual: /content/OpenVoice

📦 Instalando dependencias del sistema...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
📦 Instalando dependencias de Python...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 42.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing meta

In [ ]:
# =============================================
# 3. DESCARGAR MODELOS DE OPENVOICE
# =============================================

print("\n" + "="*60)
print("3. DESCARGANDO MODELOS DE OPENVOICE")
print("="*60)

model_urls = {
    'openvoice_v1': 'https://myshell-public-repo-host.s3.amazonaws.com/openvoice/checkpoints_1226.zip',
    'openvoice_v2': 'https://myshell-public-repo-host.s3.amazonaws.com/openvoice/checkpoints_v2_0417.zip'
}

def download_and_extract(url):
    """Descarga y extrae un archivo ZIP, verifica si ya existe"""
    zip_file = os.path.basename(url)
    extract_dir = os.path.splitext(zip_file)[0]

    # 1. Verificar si ya está extraído y parece completo
    if os.path.exists(extract_dir):
        # Verificar que tenga contenido
        content = list(os.listdir(extract_dir))
        if len(content) > 0:
            print(f"✅ {extract_dir} ya existe ({len(content)} elementos)")
            return True
        else:
            print(f"⚠️  {extract_dir} existe pero está vacío, reintentando...")
            !rm -rf $extract_dir

    # 2. Verificar si el ZIP ya existe y es válido
    zip_exists = False
    if os.path.exists(zip_file):
        # Verificar tamaño mínimo (1MB)
        file_size = os.path.getsize(zip_file) / (1024 * 1024)  # MB
        if file_size > 1:
            print(f"📦 {zip_file} ya existe ({file_size:.1f} MB)")
            zip_exists = True
        else:
            print(f"⚠️  {zip_file} existe pero es muy pequeño ({file_size:.1f} MB), redescargando...")
            !rm -f $zip_file

    # 3. Descargar si es necesario
    if not zip_exists:
        print(f"📥 Descargando {zip_file}...")
        !wget -q $url
        if not os.path.exists(zip_file):
            print(f"❌ Error: No se pudo descargar {zip_file}")
            return False

    # 4. Extraer
    if os.path.exists(zip_file):
        print(f"📦 Descomprimiendo {zip_file}...")
        os.makedirs(extract_dir, exist_ok=True)
        result = os.system(f'unzip -q {zip_file} -d {extract_dir} 2>/dev/null')

        if result == 0:
            print(f"✅ {zip_file} extraído exitosamente")

            # Verificar contenido extraído
            content = list(os.listdir(extract_dir))
            print(f"   📁 Contenido: {len(content)} elementos")

            # Opcional: eliminar ZIP para ahorrar espacio
            # !rm -f $zip_file
            return True
        else:
            print(f"❌ Error al extraer {zip_file}")
            return False

    return False

# Procesar cada modelo
for name, url in model_urls.items():
    print(f"\n🔧 Procesando {name}...")
    download_and_extract(url)

print("\n" + "="*60)
print("✅ PROCESO DE DESCARGA COMPLETADO")
print("="*60)


3. DESCARGANDO MODELOS DE OPENVOICE

🔧 Procesando openvoice_v1...
📥 Descargando checkpoints_1226.zip...
📦 Descomprimiendo checkpoints_1226.zip...
✅ checkpoints_1226.zip extraído exitosamente
   📁 Contenido: 1 elementos

🔧 Procesando openvoice_v2...
📥 Descargando checkpoints_v2_0417.zip...
📦 Descomprimiendo checkpoints_v2_0417.zip...
✅ checkpoints_v2_0417.zip extraído exitosamente
   📁 Contenido: 1 elementos

✅ PROCESO DE DESCARGA COMPLETADO


In [ ]:
# =============================================
# 4. ORGANIZAR ESTRUCTURA DE CHECKPOINTS
# =============================================

print("\n" + "="*60)
print("4. ORGANIZANDO CHECKPOINTS")
print("="*60)

# Crear estructura de directorios
os.makedirs('checkpoints/base_speakers', exist_ok=True)
os.makedirs('checkpoints/converter', exist_ok=True)
os.makedirs('checkpoints_v2/base_speakers', exist_ok=True)
os.makedirs('checkpoints_v2/converter', exist_ok=True)

# Función para copiar directorios completos
def copy_dir_contents(src_dir, dst_dir):
    """Copia todo el contenido de src_dir a dst_dir"""
    if os.path.exists(src_dir):
        # Usar cp -r para copiar recursivamente
        os.system(f'cp -r {src_dir}/* {dst_dir}/ 2>/dev/null || true')
        return True
    return False

# Copiar directorios completos (no solo archivos sueltos)
print("Copiando checkpoints V1...")
if os.path.exists('checkpoints_1226/checkpoints/base_speakers'):
    # Copiar cada subdirectorio individualmente
    for item in os.listdir('checkpoints_1226/checkpoints/base_speakers'):
        src = os.path.join('checkpoints_1226/checkpoints/base_speakers', item)
        dst = os.path.join('checkpoints/base_speakers', item)
        if os.path.isdir(src):
            os.system(f'cp -r "{src}" "{dst}" 2>/dev/null || true')

if os.path.exists('checkpoints_1226/checkpoints/converter'):
    for item in os.listdir('checkpoints_1226/checkpoints/converter'):
        src = os.path.join('checkpoints_1226/checkpoints/converter', item)
        dst = os.path.join('checkpoints/converter', item)
        os.system(f'cp -r "{src}" "{dst}" 2>/dev/null || true')

print("Copiando checkpoints V2...")
if os.path.exists('checkpoints_v2_0417/checkpoints_v2/base_speakers'):
    # Para V2, copiar recursivamente toda la estructura
    os.system('cp -r checkpoints_v2_0417/checkpoints_v2/base_speakers/* checkpoints_v2/base_speakers/ 2>/dev/null || true')

if os.path.exists('checkpoints_v2_0417/checkpoints_v2/converter'):
    os.system('cp -r checkpoints_v2_0417/checkpoints_v2/converter/* checkpoints_v2/converter/ 2>/dev/null || true')

# Verificar que los archivos se copiaron correctamente
print("\n✅ Checkpoints copiados. Verificando estructura...")

print("\n📁 Estructura de checkpoints V1:")
!find checkpoints -type f | sort

print("\n📁 Estructura de checkpoints V2:")
!find checkpoints_v2 -type f | sort

# Opcional: Limpiar archivos temporales solo si la copia fue exitosa
print("\n🧹 Limpiando archivos temporales...")
if os.path.exists('checkpoints/base_speakers/EN') and os.path.exists('checkpoints_v2/converter'):
    !rm -rf checkpoints_1226 checkpoints_v2_0417 2>/dev/null || echo "No se pudieron eliminar todos los temporales"
    print("✅ Archivos temporales eliminados")
else:
    print("⚠️  No se eliminaron temporales porque la copia puede no estar completa")


4. ORGANIZANDO CHECKPOINTS
Copiando checkpoints V1...
Copiando checkpoints V2...

✅ Checkpoints copiados. Verificando estructura...

📁 Estructura de checkpoints V1:
checkpoints/base_speakers/EN/checkpoint.pth
checkpoints/base_speakers/EN/config.json
checkpoints/base_speakers/EN/en_default_se.pth
checkpoints/base_speakers/EN/en_style_se.pth
checkpoints/base_speakers/ZH/checkpoint.pth
checkpoints/base_speakers/ZH/config.json
checkpoints/base_speakers/ZH/zh_default_se.pth
checkpoints/converter/checkpoint.pth
checkpoints/converter/config.json

📁 Estructura de checkpoints V2:
checkpoints_v2/base_speakers/ses/en-au.pth
checkpoints_v2/base_speakers/ses/en-br.pth
checkpoints_v2/base_speakers/ses/en-default.pth
checkpoints_v2/base_speakers/ses/en-india.pth
checkpoints_v2/base_speakers/ses/en-newest.pth
checkpoints_v2/base_speakers/ses/en-us.pth
checkpoints_v2/base_speakers/ses/es.pth
checkpoints_v2/base_speakers/ses/fr.pth
checkpoints_v2/base_speakers/ses/jp.pth
checkpoints_v2/base_speakers/se

In [ ]:
# =============================================
# 5. VERIFICAR INSTALACIÓN DE OPENVOICE
# =============================================

print("\n" + "="*60)
print("5. VERIFICANDO INSTALACIÓN DE OPENVOICE")
print("="*60)

modules = [
    "librosa", "av", "faster_whisper", "pydub", "wavmark", "numpy",
    "eng_to_ipa", "inflect", "unidecode", "whisper_timestamped",
    "openai", "dotenv", "pypinyin", "cn2an", "jieba", "gradio", "langid", "soundfile", "soxr"
]

all_ok = True
for m in modules:
    try:
        __import__(m.replace("-", "_"))
        print(f"✅ {m}")
    except ImportError as e:
        print(f"❌ {m} - Error: {str(e)[:50]}...")
        all_ok = False

if all_ok:
    print("\n🎉 ¡Todos los módulos de OpenVoice están instalados!")
else:
    print("\n⚠️  Algunos módulos tienen problemas. Ejecuta nuevamente la celda de instalación.")


5. VERIFICANDO INSTALACIÓN DE OPENVOICE
✅ librosa
✅ av
✅ faster_whisper
✅ pydub
✅ wavmark
✅ numpy
✅ eng_to_ipa
✅ inflect
✅ unidecode
Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

✅ whisper_timestamped
✅ openai
✅ dotenv
✅ pypinyin
✅ cn2an
✅ jieba
✅ gradio
✅ langid
✅ soundfile
✅ soxr

🎉 ¡Todos los módulos de OpenVoice están instalados!


In [ ]:
# =============================================
# 6. INSTALACIÓN DE MELOTTS
# =============================================

print("\n" + "="*60)
print("6. INSTALANDO MELOTTS")
print("="*60)

# Volver al directorio base
os.chdir(BASE_DIR)

# Clonar MeloTTS si no existe
if not (MELOTTS_DIR / '.git').exists():
    print("📥 Clonando MeloTTS...")
    !git clone https://github.com/PhonelixR/MeloTTS.git "{MELOTTS_DIR}" --quiet
else:
    print("✅ MeloTTS ya está clonado")

os.chdir(MELOTTS_DIR)
print(f"📂 Directorio actual: {os.getcwd()}")

# Instalar dependencias de MeloTTS
print("\n📦 Instalando dependencias de MeloTTS...")
!pip install -r requirements.txt -q
!pip install -q -e .

# Descargar diccionario para japonés
print("\n🗾 Descargando diccionario japonés...")
!python -m unidic download


6. INSTALANDO MELOTTS
📥 Clonando MeloTTS...
📂 Directorio actual: /content/MeloTTS

📦 Instalando dependencias de MeloTTS...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 93.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 93.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━

In [ ]:
# =============================================
# 7. DESCARGAR DATOS DE NLTK (IMPORTANTE PARA MELOTTS)
# =============================================

print("\n" + "="*60)
print("7. CONFIGURANDO NLTK")
print("="*60)

import nltk
import ssl

# Configurar SSL para evitar errores
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Descargar recursos necesarios
resources = ['averaged_perceptron_tagger', 'punkt', 'cmudict', 'averaged_perceptron_tagger_eng']

for resource in resources:
    try:
        nltk.download(resource, quiet=True)
        print(f"✅ {resource}")
    except:
        print(f"⚠️  Error descargando {resource}")

print("✅ Recursos NLTK configurados")


7. CONFIGURANDO NLTK
✅ averaged_perceptron_tagger
✅ punkt
✅ cmudict
✅ averaged_perceptron_tagger_eng
✅ Recursos NLTK configurados


In [ ]:
# =============================================
# 8. VERIFICACIÓN FINAL DE INSTALACIÓN
# =============================================

print("\n" + "="*60)
print("8. VERIFICACIÓN FINAL")
print("="*60)

import torch
import librosa

print("📊 Versiones instaladas:")
print(f"   PyTorch: {torch.__version__}")
print(f"   Librosa: {librosa.__version__}")
print(f"   CUDA disponible: {torch.cuda.is_available()}")

print("\n📁 Verificación de rutas:")
print(f"   OpenVoice: {'✅' if OPENVOICE_DIR.exists() else '❌'}")
print(f"   MeloTTS: {'✅' if MELOTTS_DIR.exists() else '❌'}")
print(f"   Checkpoints V1: {'✅' if (OPENVOICE_DIR / 'checkpoints').exists() else '❌'}")
print(f"   Checkpoints V2: {'✅' if (OPENVOICE_DIR / 'checkpoints_v2').exists() else '❌'}")


8. VERIFICACIÓN FINAL
📊 Versiones instaladas:
   PyTorch: 2.9.0+cu126
   Librosa: 0.11.0
   CUDA disponible: False

📁 Verificación de rutas:
   OpenVoice: ✅
   MeloTTS: ✅
   Checkpoints V1: ✅
   Checkpoints V2: ✅


In [ ]:
# =============================================
# 9. PREPARAR DEMOS DE OPENVOICE (CON MENÚ INTERACTIVO)
# =============================================

print("\n" + "="*60)
print("9. PREPARANDO DEMOS DE OPENVOICE")
print("="*60)

os.chdir(OPENVOICE_DIR)

# Convertir notebooks a scripts
print("🔄 Convirtiendo notebooks a scripts Python...")
!jupyter nbconvert --to python demo_part1.ipynb 2>/dev/null || echo "demo_part1 ya convertido"
!jupyter nbconvert --to python demo_part2.ipynb 2>/dev/null || echo "demo_part2 ya convertido"
!jupyter nbconvert --to python demo_part3.ipynb 2>/dev/null || echo "demo_part3 ya convertido"

print("\n✅ Demos convertidos:")
for demo in ['demo_part1.py', 'demo_part2.py', 'demo_part3.py']:
    if os.path.exists(demo):
        print(f"   📄 {demo}")


9. PREPARANDO DEMOS DE OPENVOICE
🔄 Convirtiendo notebooks a scripts Python...

✅ Demos convertidos:
   📄 demo_part1.py
   📄 demo_part2.py
   📄 demo_part3.py


In [ ]:
# =============================================
# 10. FUNCIÓN PARA EJECUTAR DEMOS CON VISUALIZACIÓN
# =============================================

def run_demo(demo_name, show_output=True):
    """Ejecuta un demo y muestra su output"""
    print(f"\n{'='*60}")
    print(f"EJECUTANDO: {demo_name}")
    print(f"{'='*60}")

    if not os.path.exists(demo_name):
        print(f"❌ Error: {demo_name} no encontrado")
        return False

    try:
        # Ejecutar el demo y capturar output
        result = subprocess.run(
            ['python', demo_name],
            capture_output=True,
            text=True,
            timeout=60  # 60 segundos máximo
        )

        if show_output:
            if result.stdout:
                print("📤 SALIDA:")
                print(result.stdout[:5000])  # Mostrar primeros 5000 caracteres
                if len(result.stdout) > 5000:
                    print(f"... (output truncado, {len(result.stdout)} caracteres total)")

            if result.stderr:
                print("⚠️  ERRORES:")
                print(result.stderr[:2000])

        print(f"\n✅ {demo_name} ejecutado (código: {result.returncode})")
        return True

    except subprocess.TimeoutExpired:
        print(f"⏰ {demo_name} excedió el tiempo límite")
        return False
    except Exception as e:
        print(f"❌ Error ejecutando {demo_name}: {str(e)}")
        return False

In [ ]:
# =============================================
# 11. MENÚ INTERACTIVO PARA EJECUTAR DEMOS
# =============================================

print("\n" + "="*60)
print("🎮 11. MENÚ INTERACTIVO DE DEMOS")
print("="*60)

while True:
    print("\nSelecciona una opción:")
    print("1. 🎤 demo_part1.py - Síntesis básica de OpenVoice")
    print("2. 🔄 demo_part2.py - Conversión de voz (requiere OpenAI API key)")
    print("3. 🚀 demo_part3.py - OpenVoice V2 (requiere MeloTTS)")
    print("4. 🧪 Probar MeloTTS (test básico)")
    print("5. 🌐 Lanzar OpenVoice Gradio App (V1 - Legacy)")
    print("6. 🌐 Lanzar OpenVoice Gradio App (V2 - Fork)")
    print("7. 🌐 Lanzar MeloTTS Gradio App")
    print("8. 📊 Mostrar verificación completa")
    print("0. ❌ Salir del menú")

    try:
        choice = input("\n👉 Tu elección (0-8): ").strip()

        if choice == '0':
            print("👋 Saliendo del menú...")
            break

        elif choice == '1':
            print("\n" + "="*60)
            print("🎤 EJECUTANDO demo_part1.py - Síntesis básica de OpenVoice")
            print("="*60)
            !python demo_part1.py 2>&1

        elif choice == '2':
            print("\n" + "="*60)
            print("🔄 EJECUTANDO demo_part2.py - Conversión de voz")
            print("="*60)
            print("⚠️  IMPORTANTE: demo_part2 requiere una API key de OpenAI")
            print("   Configura tu variable de entorno OPENAI_API_KEY o edita el archivo")
            proceed = input("   ¿Continuar de todos modos? (s/n): ").strip().lower()
            if proceed == 's':
                !python demo_part2.py 2>&1

        elif choice == '3':
            print("\n" + "="*60)
            print("🚀 EJECUTANDO demo_part3.py - OpenVoice V2")
            print("="*60)
            print("⚠️  IMPORTANTE: demo_part3 requiere MeloTTS para funcionar")
            proceed = input("   ¿Continuar? (s/n): ").strip().lower()
            if proceed == 's':
                !python demo_part3.py 2>&1

        elif choice == '4':
            print("\n" + "="*60)
            print("🧪 EJECUTANDO Prueba básica de MeloTTS")
            print("="*60)
            os.chdir(MELOTTS_DIR)
            if (MELOTTS_DIR / 'test').exists():
                os.chdir('test')
                print("📂 Directorio de tests encontrado")
                print("\n🎯 Ejecutando test_base_model_tts_package.py ES...")
                !python test_base_model_tts_package.py ES 2>&1
            else:
                print("❌ No se encontró el directorio de tests")
            os.chdir(OPENVOICE_DIR)  # Volver a OpenVoice

        elif choice == '5':
            print("\n" + "="*60)
            print("🌐 LANZANDO OpenVoice Gradio App (V1 - Legacy)")
            print("="*60)
            print("⚠️  Esta app se ejecutará en segundo plano")
            print("   Busca el enlace 'Running on public URL' en el output")
            print("   Presiona Ctrl+C para detenerla\n")
            !python -m openvoice.openvoice_app --share 2>&1

        elif choice == '6':
            print("\n" + "="*60)
            print("🌐 LANZANDO OpenVoice Gradio App (V2 - Fork)")
            print("="*60)
            print("⚠️  Esta app se ejecutará en segundo plano")
            print("   Busca el enlace 'Running on public URL' en el output")
            print("   Presiona Ctrl+C para detenerla\n")
            !python -m openvoice.openvoice_app_v2 --share 2>&1

        elif choice == '7':
            print("\n" + "="*60)
            print("🌐 LANZANDO MeloTTS Gradio App")
            print("="*60)
            os.chdir(MELOTTS_DIR / 'melo')
            print("⚠️  Esta app se ejecutará en segundo plano")
            print("   Busca el enlace 'Running on public URL' en el output")
            print("   Presiona Ctrl+C para detenerla\n")
            !python -m app --share 2>&1
            os.chdir(OPENVOICE_DIR)  # Volver a OpenVoice

        elif choice == '8':
            # Verificación completa
            print("\n" + "="*60)
            print("📊 VERIFICACIÓN COMPLETA DEL SISTEMA")
            print("="*60)

            # Verificar módulos críticos
            critical_modules = ['torch', 'librosa', 'gradio', 'soundfile', 'transformers']
            for mod in critical_modules:
                try:
                    __import__(mod)
                    print(f"✅ {mod}")
                except:
                    print(f"❌ {mod}")

            # Verificar archivos importantes
            important_files = [
                ('checkpoints/base_speakers/EN/checkpoint.pth', 'Modelo EN V1'),
                ('checkpoints_v2/converter/checkpoint.pth', 'Converter V2'),
                ('openvoice/api.py', 'API OpenVoice'),
                (MELOTTS_DIR / 'melo/api.py', 'API MeloTTS')
            ]

            for path, desc in important_files:
                if isinstance(path, Path):
                    exists = path.exists()
                else:
                    exists = os.path.exists(path)
                print(f"{'✅' if exists else '❌'} {desc}")

        else:
            print("❌ Opción no válida. Intenta de nuevo.")

    except KeyboardInterrupt:
        print("\n\n⚠️  Interrumpido por el usuario")
        break
    except Exception as e:
        print(f"❌ Error: {str(e)}")


🎮 11. MENÚ INTERACTIVO DE DEMOS

Selecciona una opción:
1. 🎤 demo_part1.py - Síntesis básica de OpenVoice
2. 🔄 demo_part2.py - Conversión de voz (requiere OpenAI API key)
3. 🚀 demo_part3.py - OpenVoice V2 (requiere MeloTTS)
4. 🧪 Probar MeloTTS (test básico)
5. 🌐 Lanzar OpenVoice Gradio App (V1 - Legacy)
6. 🌐 Lanzar OpenVoice Gradio App (V2 - Fork)
7. 🌐 Lanzar MeloTTS Gradio App
8. 📊 Mostrar verificación completa
0. ❌ Salir del menú

👉 Tu elección (0-8): 0
👋 Saliendo del menú...


In [ ]:
# =============================================
# 12. RESUMEN FINAL Y COMANDOS ÚTILES
# =============================================

print("\n" + "="*60)
print("🎉 12. INSTALACIÓN COMPLETADA")
print("="*60)

print("""
✅ **RESUMEN DE LA INSTALACIÓN:**

📁 **OpenVoice:** /content/OpenVoice
   ├── checkpoints/         (V1: EN, ZH)
   ├── checkpoints_v2/      (V2: multi-idioma)
   └── openvoice/           (Código fuente)

📁 **MeloTTS:** /content/MeloTTS
   └── melo/                (Código fuente)

🚀 **COMANDOS ÚTILES:**

# Para OpenVoice:
cd /content/OpenVoice
python -m openvoice.openvoice_app --share           # App V1
python -m openvoice.openvoice_app_v2 --share        # App V2

# Para MeloTTS:
cd /content/MeloTTS/melo
python -m app --share                              # App MeloTTS

# Para probar manualmente:
python demo_part1.py                               # Demo básico
python demo_part3.py                               # Demo V2 (requiere MeloTTS)

🔧 **SOLUCIÓN DE PROBLEMAS:**

1. Si Colab se reinicia:
   - Vuelve a ejecutar las celdas de instalación
   - Las rutas ya están configuradas para resistir reinicios

2. Si faltan módulos:
   - Ejecuta: !pip install -r requirements.txt

3. Para probar MeloTTS:
   cd /content/MeloTTS/test
   python test_base_model_tts_package.py ES

📝 **NOTAS:**
- demo_part2.py requiere API key de OpenAI
- demo_part3.py requiere que MeloTTS esté instalado
- Las apps Gradio generan enlaces públicos que expiran
""")

print("\n" + "="*60)
print("¡Todo listo! Usa el menú interactivo para probar las funciones.")
print("="*60)


🎉 12. INSTALACIÓN COMPLETADA

✅ **RESUMEN DE LA INSTALACIÓN:**

📁 **OpenVoice:** /content/OpenVoice
   ├── checkpoints/         (V1: EN, ZH)
   ├── checkpoints_v2/      (V2: multi-idioma)
   └── openvoice/           (Código fuente)

📁 **MeloTTS:** /content/MeloTTS
   └── melo/                (Código fuente)

🚀 **COMANDOS ÚTILES:**

# Para OpenVoice:
cd /content/OpenVoice
python -m openvoice.openvoice_app --share           # App V1
python -m openvoice.openvoice_app_v2 --share        # App V2

# Para MeloTTS:
cd /content/MeloTTS/melo
python -m app --share                              # App MeloTTS

# Para probar manualmente:
python demo_part1.py                               # Demo básico
python demo_part3.py                               # Demo V2 (requiere MeloTTS)

🔧 **SOLUCIÓN DE PROBLEMAS:**

1. Si Colab se reinicia:
   - Vuelve a ejecutar las celdas de instalación
   - Las rutas ya están configuradas para resistir reinicios

2. Si faltan módulos:
   - Ejecuta: !pip install -r requ

In [ ]:
# =============================================================================
# 13. RUTAS Y ARCHIVOS (ARBOL DE FICHEROS)
# =============================================================================

# =============================================================================
# ESTRUCTURA DE ARCHIVOS DE OPENVOICE
# =============================================================================
print("=" * 70)
print("ESTRUCTURA DE ARCHIVOS DE OPENVOICE")
print("=" * 70)
!find /content/OpenVoice -type f -name "*.py" -o -name "*.pth" -o -name "*.json" | sort

print("\n" + "=" * 70)
print("ESTRUCTURA DE DIRECTORIOS DE OPENVOICE")
print("=" * 70)
!find /content/OpenVoice -type d | sort

# =============================================================================
# ESTRUCTURA DE ARCHIVOS DE MELOTTS
# =============================================================================
print("\n" + "=" * 70)
print("ESTRUCTURA DE ARCHIVOS DE MELOTTS")
print("=" * 70)
!find /content/MeloTTS -type f -name "*.py" -o -name "*.pth" -o -name "*.json" | sort

print("\n" + "=" * 70)
print("ESTRUCTURA DE DIRECTORIOS DE MELOTTS")
print("=" * 70)
!find /content/MeloTTS -type d | sort

ESTRUCTURA DE ARCHIVOS DE OPENVOICE
/content/OpenVoice/checkpoints/base_speakers/EN/checkpoint.pth
/content/OpenVoice/checkpoints/base_speakers/EN/config.json
/content/OpenVoice/checkpoints/base_speakers/EN/en_default_se.pth
/content/OpenVoice/checkpoints/base_speakers/EN/en_style_se.pth
/content/OpenVoice/checkpoints/base_speakers/ZH/checkpoint.pth
/content/OpenVoice/checkpoints/base_speakers/ZH/config.json
/content/OpenVoice/checkpoints/base_speakers/ZH/zh_default_se.pth
/content/OpenVoice/checkpoints/converter/checkpoint.pth
/content/OpenVoice/checkpoints/converter/config.json
/content/OpenVoice/checkpoints_v2/base_speakers/ses/en-au.pth
/content/OpenVoice/checkpoints_v2/base_speakers/ses/en-br.pth
/content/OpenVoice/checkpoints_v2/base_speakers/ses/en-default.pth
/content/OpenVoice/checkpoints_v2/base_speakers/ses/en-india.pth
/content/OpenVoice/checkpoints_v2/base_speakers/ses/en-newest.pth
/content/OpenVoice/checkpoints_v2/base_speakers/ses/en-us.pth
/content/OpenVoice/checkpoint